# DTSC670: Foundations of Machine Learning Models
## Assignment 4: Custom Transformer and Transformation Pipeline

#### Name:

### CodeGrade
Note that this assignment will be automatically graded through CodeGrade and you will have unlimited submission attempts.  When submitting to CodeGrade, your notebook should be named `assignment4.ipynb` and there should be no errors in the file or CodeGrade will not be able to grade it.  Before submitting, I suggest that you restart your kernel and attempt to run all cells again to ensure that there will be no errors when CodeGrade runs your script.

### Details
Your task in this assignment is to create a custom transformation pipeline that takes in raw data and returns fully prepared, clean data that is ready for model training.  However, we will not actually train any models in this assignment.  This pipeline will employ an imputer class, a user-defined transformer class, and a data-normalization class.

Please note that the order of features in the final feature matrix must be correct.  See the below figure that illustrates the input and output of the transformation pipeline.  The positions of features $x_1$ and $x_2$ do not change - they remain in the first and second columns, respectively, both before and after the transformation pipeline.  In the transformed dataset, the $x_5$ feature is next, and is followed by the newly computed feature $x_6$.  Finally, the last two columns are the remaining one-hot vectors obtained from encoding the categorical feature $x_3$.

<img src="DataTransformation.png " width ="500" />

# Import Data

- Make sure that your notebook and data file are located in the same folder since this is what CodeGrade will be expecting.  
- Import the data from the file called `CustomTransformerData.csv` and call this DataFrame `custom_transform`.  
- Output the `custom_transform` DataFrame to see the data.

In [1]:
### ENTER CODE HERE ###
# Common imports
import numpy as np
import pandas as pd

#Import data from Excel
custom_transform = pd.read_csv('CustomTransformerData.csv')


# Create Numeric and Categorical DataFrames

- Create two new DataFrames: 
  - Create one DataFrame called `data_num` that holds the numeric features from the `custom_transform` DataFrame.  
  - Create another DataFrame called `data_cat` that holds the categorical feature from `custom_transform`.

In [2]:
### ENTER CODE HERE ###

data_num = custom_transform[['x1', 'x2', 'x4', 'x5']]

data_cat = custom_transform[['x3']]


In [125]:
### ENTER CODE HERE ###



# Create Custom Transformer

Create a custom transformer, just as we did in the lecture video entitled "Custom Transformers", that performs two computations: 

1. Adds an attribute to the end of the numerical data (i.e. new last column) that is equal to $\frac{x_1^3}{x_5}$ for each observation.  In other words, for each instance, you will cube the $x_1$ column and then divide by the $x_5$ column.

2. Drops the entire $x_4$ feature column if the passed function argument `drop_x4` is `True` and doesn't drop the column if `drop_x4` is `False`. (See further instructions below.)

You must name your custom transformer class `Assignment4Transformer`. Your class should include an input parameter called `drop_x4` with a default value of `True` that deletes the $x_4$ feature column when its value is `True`, but preserves the $x_4$ feature column when you pass a value of `False`.

This transformer will be used in a pipeline. In that pipeline, an imputer will be run *before* this transformer. Keep in mind that the imputer will output an array, so **this transformer must be written to accept an array.**  This is very important and a cause of many errors that students encounter.  In other words, think about using NumPy in your transformer instead of Pandas.

Additionally, this transformer will ONLY be given the numerical features of the data. The categorical feature will be handled elsewhere in the full pipeline. This means that your code for this transformer **must reflect the absence of the categorical $x_3$ column** when indexing data structures.  Again this is very important and a cause of the second most number of errors that students encounter.

In [25]:
### ENTER CODE HERE ###
from sklearn.base import BaseEstimator, TransformerMixin

class Assignment4Transformer(BaseEstimator, TransformerMixin):
    def __init__(self, drop_x4=True):
        self.drop_x4 = drop_x4       
    def fit(self, X, y=None):
        return self   
    def transform(self, X):
        x6 = (X[:,0]**3) / (X[:,3])
        if self.drop_x4 == True:
            X = np.delete(X, 2, axis=1)
        return np.c_[X,x6]

# Create Transformation Pipeline for Numerical Features

Create a custom transformation pipeline for only the numeric data called `num_pipeline` that:

1. Applies the `SimpleImputer` class to the data, where the strategy is set to `mean`.  The name of this step should be "imputer".

2. Applies the custom `Assignment4Transformer` class to the data.  Make sure that your custom transformer uses the default argument where you drop the $x_4$ column.  The name of this step should be "custom_trans".

3. Applies the `StandardScaler` class to the data.  The name of this step should be "std_scaler".

In [26]:
### ENTER CODE HERE ###
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([('imputer', SimpleImputer(strategy='mean')), 
                        ('custom_trans', Assignment4Transformer()),
                        ('std_scaler', StandardScaler())])

# Quick Testing

The full pipeline will be implemented with a `ColumnTransformer` class.  However, to be sure that our numeric pipeline is working properly, lets invoke the `fit_transform()` method of the `num_pipeline` object passing it your `data_num` DataFrame.  Save this output data into a variable called `data_num_trans`.

### Run Pipeline and Create Transformed Numeric Data

In [27]:
### ENTER CODE HERE ###
data_num_trans = num_pipeline.fit_transform(data_num)



### One-Hot Encode Categorical Features

Similarly, you will employ a `OneHotEncoder` class in the `ColumnTransformer` below to construct the final full pipeline.  However, let's instantiate an object of the `OneHotEncoder` class to use in the `ColumnTransformer`.

1. Call this object `cat_encoder` that has the `drop` parameter set to `first`.  
2. Also, include `sparse=False` when instantiating the object to make the auto grading work correctly.  
3. Next, call the `fit_transform()` method and pass it your categorical data (`data_cat`).  
4. Save this output data into a variable called `data_cat_OHE`.

In [28]:
### ENTER CODE HERE ###
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder(drop='first', sparse=False)
data_cat_OHE = cat_encoder.fit_transform(data_cat)

# Put it All Together with a Column Transformer

Now, we are finally ready to construct the full transformation pipeline called `full_pipeline` that will transform our raw data into clean, ready-to-train data.  
1. Construct this `ColumnTransformer` below.  Note that you will use:
  - the `num_pipeline` in the full_pipeline with your numeric data
  - the `cat_encoder` object in the full_pipeline with your categorical data
2. Call the `fit_transform()` method on the original `custom_transform` data to obtain the final, clean data.  
3. Save this output data into a variable called `data_trans`.

In [29]:
### ENTER CODE HERE ###

from sklearn.compose import ColumnTransformer

num_attribs = list(data_num)
cat_attribs = list(data_cat)
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ('cat', cat_encoder, cat_attribs)
])

data_trans = full_pipeline.fit_transform(custom_transform)

# Prepare for Grading

Prepare your `data_trans` NumPy array for grading by running the code below.  We are using the NumPy [around()](https://numpy.org/doc/stable/reference/generated/numpy.around.html) function to round all the values to 2 decimal places and this will return a NumPy array.

Please double check that the final order of the features in this `data_trans` array matches what was given to you at the top of this document.

In [31]:

data_trans = np.around(data_trans, decimals=2)

data_trans

array([[-1.64, -1.73, -1.2 , -1.59,  0.  ,  0.  ],
       [-1.45, -1.53, -1.16, -1.4 ,  0.  ,  1.  ],
       [-1.25, -1.38, -1.1 , -1.21,  0.  ,  0.  ],
       [-1.06,  0.  , -1.03, -1.02,  0.  ,  0.  ],
       [-0.87, -0.99, -0.93, -0.83,  0.  ,  1.  ],
       [-0.67, -0.7 , -0.82, -0.64,  0.  ,  1.  ],
       [-0.48, -0.53, -0.69, -0.45,  1.  ,  0.  ],
       [-0.29, -0.28, -0.54, -0.26,  0.  ,  0.  ],
       [-0.1 , -0.04,  0.  , -0.61,  0.  ,  1.  ],
       [ 0.1 ,  0.13, -0.18,  0.13,  1.  ,  0.  ],
       [ 0.29,  0.27,  0.03,  0.32,  0.  ,  1.  ],
       [ 0.48,  0.45,  0.26,  0.51,  0.  ,  1.  ],
       [ 0.67,  0.76,  0.5 ,  0.7 ,  0.  ,  0.  ],
       [ 0.87,  0.88,  0.76,  0.89,  1.  ,  0.  ],
       [ 1.06,  0.  ,  1.05,  1.08,  1.  ,  0.  ],
       [ 1.25,  1.42,  1.35,  1.27,  0.  ,  1.  ],
       [ 1.45,  1.55,  1.67,  1.46,  1.  ,  0.  ],
       [ 1.64,  1.71,  2.01,  1.65,  1.  ,  0.  ]])